# 한글 검색 최적화: Kiwi 형태소 분석기와 BM25 Retriever

## 개요

한글 텍스트 검색에서는 언어의 복잡한 형태 변화로 인해 정확한 검색이 어려운 문제가 있습니다. 예를 들어 "먹다"라는 동사는 "먹는다", "먹었다", "먹을 것이다" 등 다양한 형태로 변화하며, 이러한 변화 형태들을 모두 고려하여 검색해야 합니다.

### 학습 목표

이 튜토리얼에서는 다음 내용을 학습합니다:

1. **형태소 분석의 원리** - 한글 문장을 의미 단위로 분해하는 방법
2. **Kiwi 형태소 분석기** - 빠르고 정확한 한글 처리를 위한 도구
3. **BM25 Retriever** - 키워드 기반 검색 알고리즘의 활용
4. **성능 비교 분석** - 다양한 형태소 분석기들의 검색 성능 평가

### 한글 검색의 중요성

| 항목 | 설명 | 효과 |
|------|------|------|
| **정확한 매칭** | "금융보험"과 "금융 보험" 모두 검색 가능 | 검색 누락 방지 |
| **처리 속도** | 사전 분석된 토큰으로 빠른 검색 수행 | 사용자 경험 향상 |
| **한글 특화** | 한글 고유의 언어적 특징 활용 | 검색 정확도 증가 |
| **실무 적용** | 실제 서비스에서 즉시 활용 가능 | 비즈니스 가치 창출 |

---

## 목차

### Part 1: 환경 설정
- API 키 설정 및 LangSmith 연동
- Kiwi 형태소 분석기 설치 및 소개

### Part 2: 형태소 분석 기초
- 형태소 분석의 개념과 필요성
- Kiwi를 활용한 토큰화 실습

### Part 3: 검색 성능 비교
- BM25 vs Kiwi-BM25 vs FAISS 성능 비교
- 다양한 Ensemble Retriever 조합 실험

### Part 4: 형태소 분석기 종합 비교
- KoNLPy 라이브러리의 다양한 분석기 활용
- Kkma, Okt, Komoran, Hannanum, Kiwi 성능 분석

---

## 환경 설정

### Kiwi 형태소 분석기 특징

**Kiwi**는 한글 자연어 처리를 위한 고성능 형태소 분석기입니다.

#### 주요 특징

| 특징 | 설명 |
|------|------|
| **고성능 처리** | C++로 구현되어 다른 분석기 대비 빠른 처리 속도 |
| **높은 정확도** | 최신 딥러닝 기술 적용으로 정확한 형태소 분석 |
| **Python 호환성** | Python 환경에서 간단한 API로 사용 가능 |
| **지속적 개선** | 활발한 커뮤니티와 정기적인 업데이트 |

#### 참고 자료

- [Kiwi 프로젝트 GitHub 저장소](https://github.com/bab2min/kiwipiepy)

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

In [3]:
# !pip install kiwipiepy

# Part 2: 형태소 분석 기초

## 형태소 분석의 개념

**형태소 분석**은 자연어 텍스트를 언어학적으로 의미있는 최소 단위인 형태소로 분해하는 과정입니다.

### 형태소의 정의

형태소(morpheme)는 의미를 가지는 언어의 최소 단위입니다. 한글에서는 다음과 같이 분류됩니다:

#### 형태소 분석 예시

```
"맛있는 피자를 먹었어요"
├── "맛있" (어근) + "는" (형용사 어미)
├── "피자" (명사)  
├── "를" (목적격 조사)
├── "먹" (동사 어근) + "었" (과거 시제) + "어요" (종결어미)
```

### 형태소 유형

| 형태소 유형 | 설명 | 예시 |
|-------------|------|------|
| **자립 형태소** | 단독으로 사용 가능한 형태소 | 명사(책, 사람), 대명사(나, 너), 수사(하나, 둘) |
| **의존 형태소** | 다른 형태소와 결합해야 사용 가능 | 조사(은/는, 이/가), 어미(-다, -요), 접사(-적, -하다) |

### 형태소 분석의 필요성

| 목적 | 설명 | 효과 |
|------|------|------|
| **의미 파악** | 문장의 정확한 의미 구조 파악 | "먹는다"와 "먹었다"의 시제 구분 |
| **검색 최적화** | 복합어를 구성 요소로 분해 | "금융보험"에서 "금융", "보험" 각각 검색 |
| **언어 처리** | 한글의 복잡한 활용 처리 | 어미 변화, 불규칙 활용 대응 |
| **응용 분야** | 다양한 NLP 작업의 기초 | 번역, 요약, 감정분석, 질의응답 |

In [ ]:
# Kiwi 형태소 분석기 import
from kiwipiepy import Kiwi

# Kiwi 분석기 초기화 (한국어 형태소 분석 모델 로드)
kiwi = Kiwi()

## Kiwi 형태소 분석 실습

### 토큰화(Tokenization) 과정

**토큰화**는 텍스트를 의미 있는 토큰 단위로 분리하는 과정입니다. Kiwi에서 각 토큰은 다음 정보를 포함합니다:

#### 토큰 속성 정보

| 속성 | 설명 | 예시 |
|------|------|------|
| **form** | 실제 토큰의 표면형 | "안녕하세요" |
| **tag** | 품사 태그 정보 | NNG(일반명사), XSA(형용사파생접미사) |
| **start** | 원본 텍스트에서의 시작 위치 | 0, 5, 10 |
| **len** | 토큰의 문자 길이 | 2, 3, 1 |

#### 주요 품사 태그

| 태그 | 의미 | 예시 |
|------|------|------|
| **NNG** | 일반명사 | 사람, 책, 컴퓨터 |
| **NNP** | 고유명사 | 서울, 김철수, 삼성 |
| **VV** | 동사 | 가다, 먹다, 보다 |
| **VA** | 형용사 | 크다, 작다, 좋다 |
| **JKS** | 주격조사 | 이, 가 |
| **JKO** | 목적격조사 | 을, 를 |

다음 셀에서 실제 토큰화 결과를 확인해보겠습니다.

In [ ]:
# 예시 문장을 토큰화하여 형태소 분석 결과 확인
kiwi.tokenize("안녕하세요? 형태소 분석기 키위입니다")

# Part 3: 검색 성능 비교

## 검색 시나리오 설계

실제 검색 환경을 시뮬레이션하기 위해 유사하지만 서로 다른 금융보험 관련 문서를 준비하였습니다. 이러한 데이터셋을 통해 각 검색 방법의 정확도와 성능을 측정할 수 있습니다.

### 테스트 데이터셋 특징

| 특성 | 설명 |
|------|------|
| **주제 통일성** | 모든 문서가 금융보험 관련 내용으로 구성 |
| **미묘한 차이** | 비슷한 용어들로 구성되어 정확한 매칭이 중요 |
| **다양한 표현** | 같은 의미를 다른 방식으로 표현한 문서 포함 |
| **노이즈 데이터** | 의도적으로 관련성이 낮은 문서 포함 |

### 비교 대상 검색 방법

| 검색 방법 | 설명 | 특징 |
|-----------|------|------|
| **기본 BM25** | 영어 기준 키워드 매칭 | 단순 문자열 매칭, 한글 특성 미고려 |
| **Kiwi-BM25** | 한글 형태소 분석 기반 BM25 | 한글 언어적 특성 반영 |
| **FAISS** | 벡터 유사도 기반 검색 | 의미적 유사성 중심 |
| **Ensemble** | 여러 방법을 조합한 하이브리드 | 키워드 + 의미 검색의 장점 결합 |

### 검색 전략 비교

| 전략 | 접근 방식 | 장점 | 단점 |
|------|-----------|------|------|
| **키워드 기반** | 정확한 단어 일치 검색 | 명확한 매칭, 빠른 처리 | 동의어/유의어 처리 한계 |
| **의미 기반** | 문맥적 유사성 검색 | 의미적 연관성 파악 | 정확한 키워드 매칭 약함 |
| **하이브리드** | 두 방식의 가중 조합 | 각 방식의 장점 활용 | 복잡성 증가, 조율 필요 |

In [ ]:
# 검색 성능 비교를 위한 라이브러리들 import
from langchain.retrievers import EnsembleRetriever  # 여러 검색기 조합
from langchain_community.retrievers import BM25Retriever  # BM25 알고리즘 검색기
from langchain_core.documents import Document  # 문서 객체
from langchain.vectorstores import FAISS  # 벡터 검색을 위한 FAISS
from langchain_openai import OpenAIEmbeddings  # OpenAI 임베딩

# 테스트용 금융보험 관련 문서들 (유사하지만 다른 상품들)
docs = [
    Document(
        page_content="금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다."
    ),
    Document(
        page_content="금융저축보험은 규칙적인 저축을 통해 목돈을 마련할 수 있으며, 생명보험 기능도 겸비하고 있습니다."
    ),
    Document(
        page_content="저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다."
    ),
    Document(
        page_content="금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다."
    ),
    Document(
        page_content="금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다."
    ),
    Document(
        page_content="금보험은 저축성과를 극대화합니다. 특히 노후 대비 저축에 유리하게 구성되어 있습니다."
    ),
    Document(
        page_content="금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요."
    ),
]

In [ ]:
# 각 문서에 대해 Kiwi 형태소 분석 결과 확인
for doc in docs:
    # 토큰화 후 form(단어 형태)만 추출하여 공백으로 연결
    print(" ".join([token.form for token in kiwi.tokenize(doc.page_content)]))

In [ ]:
# Kiwi를 이용한 토큰화 함수 정의
def kiwi_tokenize(text):
    """
    한글 텍스트를 Kiwi로 형태소 분석하여 토큰 리스트 반환
    Args:
        text: 분석할 텍스트
    Returns:
        list: 토큰화된 단어들의 리스트
    """
    return [token.form for token in kiwi.tokenize(text)]

## 검색기 구성 및 전략

### 검색기 구성 전략

다음 7가지 검색 전략을 구성하여 성능을 비교합니다:

#### 단일 검색기

| 검색기 | 설명 | 특징 |
|--------|------|------|
| **BM25** | 기본 BM25 알고리즘 | 영어 토큰화 기준, 단순 키워드 매칭 |
| **Kiwi-BM25** | 한글 형태소 분석 적용 BM25 | 한글 언어적 특성을 고려한 토큰화 |
| **FAISS** | 벡터 유사도 검색 | 의미적 유사성 기반 검색 |

#### Ensemble 검색기 (가중치 조합)

| 검색기 | 조합 | 가중치 | 특징 |
|--------|------|--------|------|
| **BM25+FAISS (7:3)** | 키워드 + 의미 | BM25 70%, FAISS 30% | 키워드 매칭 우선 |
| **BM25+FAISS (3:7)** | 키워드 + 의미 | BM25 30%, FAISS 70% | 의미 검색 우선 |
| **Kiwi-BM25+FAISS (7:3)** | 한글키워드 + 의미 | Kiwi 70%, FAISS 30% | 한글 최적화 키워드 우선 |
| **Kiwi-BM25+FAISS (3:7)** | 한글키워드 + 의미 | Kiwi 30%, FAISS 70% | 의미 검색 + 한글 고려 |

### MMR(Maximal Marginal Relevance)

**MMR**은 검색 결과의 품질을 향상시키기 위한 알고리즘입니다:

#### MMR의 특징

| 항목 | 설명 |
|------|------|
| **목적** | 검색 결과의 관련성과 다양성을 동시에 고려 |
| **관련성** | 쿼리와 높은 유사도를 가진 문서 선택 |
| **다양성** | 이미 선택된 문서와 중복성을 최소화 |
| **효과** | 더 풍부하고 포괄적인 검색 결과 제공 |

#### MMR 계산 공식

```
MMR = λ × Sim(q, d) - (1-λ) × max Sim(d, d')
```

- `λ`: 관련성과 다양성의 균형 조절 매개변수
- `Sim(q, d)`: 쿼리와 문서간 유사도
- `Sim(d, d')`: 선택된 문서들 간의 유사도

In [ ]:
# === 단일 검색기 설정 ===

# 1. 기본 BM25 검색기 (영어 토큰화 기준)
bm25 = BM25Retriever.from_documents(docs)

# 2. Kiwi 형태소 분석을 사용하는 BM25 검색기 (한글 최적화)
kiwi_bm25 = BM25Retriever.from_documents(docs, preprocess_func=kiwi_tokenize)

# 3. FAISS 벡터 검색기 (의미 기반 검색)
faiss = FAISS.from_documents(
    docs, 
    OpenAIEmbeddings(model="text-embedding-3-small")
).as_retriever()

# === 앙상블 검색기 설정 ===

# 4. BM25 + FAISS (7:3 비율) - 키워드 검색 우선
bm25_faiss_73 = EnsembleRetriever(
    retrievers=[bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # BM25에 70%, FAISS에 30% 가중치 부여
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식 사용
)

# 5. BM25 + FAISS (3:7 비율) - 의미 검색 우선  
bm25_faiss_37 = EnsembleRetriever(
    retrievers=[bm25, faiss],
    weights=[0.3, 0.7],  # BM25에 30%, FAISS에 70% 가중치 부여
    search_type="mmr",
)

# 6. Kiwi-BM25 + FAISS (7:3 비율) - 한글 키워드 검색 우선
kiwibm25_faiss_73 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],  # 한글 최적화된 BM25 + FAISS 조합
    weights=[0.7, 0.3],
    search_type="mmr",
)

# 7. Kiwi-BM25 + FAISS (3:7 비율) - 의미 검색 우선이지만 한글 고려
kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.3, 0.7],  # 의미 검색을 우선시하되 한글 특성 반영
    search_type="mmr",
)

# 모든 검색기를 딕셔너리로 정리 (비교 실험용)
retrievers = {
    "bm25": bm25,
    "kiwi_bm25": kiwi_bm25, 
    "faiss": faiss,
    "bm25_faiss_73": bm25_faiss_73,
    "bm25_faiss_37": bm25_faiss_37,
    "kiwi_bm25_faiss_73": kiwibm25_faiss_73,
    "kiwi_bm25_faiss_37": kiwibm25_faiss_37,
}

In [ ]:
# 검색 결과를 비교하여 출력하는 함수 정의
def print_search_results(retrievers, query):
    """
    여러 검색기의 결과를 비교하여 출력하는 함수
    Args:
        retrievers: 검색기들의 딕셔너리 
        query: 검색할 쿼리 문자열
    """
    print(f"Query: {query}")
    
    # 각 검색기별로 검색을 수행하고 첫 번째 결과 출력
    for name, retriever in retrievers.items():
        # 검색 수행 후 첫 번째 결과의 내용 추출
        search_result = retriever.invoke(query)[0].page_content
        print(f"{name}    \t: {search_result}")
    
    print("===" * 20)  # 구분선 출력

## 검색 성능 테스트

### 테스트 시나리오 설계

6가지 서로 다른 검색 쿼리를 통해 각 검색기의 성능을 평가합니다. 각 쿼리는 한글 검색에서 발생할 수 있는 다양한 도전 과제를 포함하고 있습니다.

#### 테스트 쿼리 분석

| 쿼리 | 유형 | 테스트 목적 |
|------|------|-------------|
| **"금융보험"** | 기본 복합어 | 표준적인 복합어 검색 성능 |
| **"금융 보험"** | 공백 포함 | 공백으로 분리된 키워드 처리 |
| **"금융저축보험"** | 긴 복합어 | 3개 이상의 형태소로 구성된 복합어 |
| **"축산물 보험"** | 특수 조합 | 일반적이지 않은 키워드 조합 |
| **"저축금융보험"** | 순서 변경 | 원본과 다른 순서의 키워드 |
| **"금융보씨 개인정보 조회"** | 노이즈 포함 | 관련성이 낮은 단어가 포함된 쿼리 |

### 성능 평가 기준

| 평가 항목 | 설명 | 중요도 |
|-----------|------|--------|
| **정확도** | 관련 문서를 올바르게 검색하는 능력 | 높음 |
| **일관성** | 다양한 쿼리에 대한 안정적인 성능 | 높음 |
| **한글 최적화** | 한글 언어 특성을 활용한 검색 능력 | 높음 |
| **노이즈 내성** | 관련성이 낮은 키워드에 대한 강건성 | 중간 |

### 예상 결과

각 검색기별로 다음과 같은 특성을 보일 것으로 예상됩니다:

| 검색기 유형 | 예상 강점 | 예상 약점 |
|-------------|-----------|-----------|
| **기본 BM25** | 정확한 키워드 매칭 | 한글 형태소 미고려 |
| **Kiwi-BM25** | 한글 최적화, 형태소 분석 활용 | 의미적 연관성 부족 |
| **FAISS** | 의미적 유사성 검색 | 정확한 키워드 매칭 약함 |
| **Ensemble** | 균형잡힌 성능 | 설정 복잡성 |

In [ ]:
# === 실전 검색 테스트 수행 ===

# 6가지 다른 쿼리로 검색 성능 비교 실험

print_search_results(retrievers, "금융보험")        # 기본 복합어 검색
print_search_results(retrievers, "금융 보험")       # 공백 분리 검색  
print_search_results(retrievers, "금융저축보험")    # 긴 복합어 검색
print_search_results(retrievers, "축산물 보험")     # 특수 상품명 검색
print_search_results(retrievers, "저축금융보험")    # 단어 순서 변경 검색
print_search_results(retrievers, "금융보씨 개인정보 조회")  # 노이즈 포함 검색

## Konlpy

# Part 4: 형태소 분석기 종합 비교

## KoNLPy 라이브러리 소개

**KoNLPy**(Korean NLP in Python)는 한글 자연어 처리를 위한 대표적인 Python 라이브러리입니다. 다양한 형태소 분석기를 통합적으로 제공하여 연구자와 개발자들이 용도에 맞는 분석기를 선택할 수 있게 합니다.

### 형태소 분석기별 특성 비교

#### 상세 분석기 특징

| 분석기 | 개발 배경 | 처리 속도 | 정확도 | 메모리 사용량 | 적용 분야 |
|--------|-----------|-----------|--------|---------------|-----------|
| **Kkma** | 서울대 언어학과 | 느림 | 매우 높음 | 많음 | 학술 연구, 정밀 분석 |
| **Okt** | 오픈소스 프로젝트 | 보통 | 높음 | 보통 | 일반 텍스트 처리 |
| **Komoran** | SHINEWARE | 보통 | 높음 | 보통 | 상업적 응용 |
| **Hannanum** | KAIST | 느림 | 높음 | 많음 | 전통적 언어학 연구 |
| **Kiwi** | 개인 프로젝트 | 매우 빠름 | 매우 높음 | 적음 | 실시간 서비스 |

### 형태소 분석 방식 차이

#### 분석 철학 및 접근법

| 분석기 | 분석 철학 | 장점 | 단점 | 권장 용도 |
|--------|-----------|------|------|-----------|
| **Kkma** | 최대한 세밀한 분해 | 정밀한 언어학적 분석 | 과도한 분할, 속도 저하 | 정확성 최우선 연구 |
| **Okt** | 균형잡힌 실용적 분석 | 안정적 성능, 널리 사용 | 때때로 부정확한 분할 | 범용 텍스트 처리 |
| **Komoran** | 사전 기반 정확한 분석 | 높은 정확도, 안정성 | 신조어 처리 약함 | 정형화된 문서 처리 |
| **Hannanum** | 전통적 언어학 방법론 | 검증된 방법론 | 현대적 표현 처리 한계 | 고전 텍스트 분석 |
| **Kiwi** | 딥러닝 기반 현대적 분석 | 빠른 속도, 높은 정확도 | 상대적으로 새로운 기술 | 실시간 대용량 처리 |

### 실무 선택 기준

#### 프로젝트 요구사항별 추천

| 요구사항 | 1순위 추천 | 2순위 추천 | 사유 |
|----------|------------|------------|------|
| **실시간 서비스** | Kiwi | Okt | 속도와 정확도의 조화 |
| **학술 연구** | Kkma | Komoran | 정밀한 분석 필요 |
| **상용 서비스** | Kiwi | Komoran | 안정성과 성능 고려 |
| **프로토타입 개발** | Okt | Kiwi | 빠른 구현과 테스트 |
| **대용량 배치 처리** | Kiwi | Okt | 처리 속도가 중요 |

In [12]:
# !pip install konlpy

In [ ]:
# KoNLPy에서 다양한 형태소 분석기들 import
from konlpy.tag import Kkma, Okt, Komoran, Hannanum
from kiwipiepy import Kiwi

# 각 분석기 초기화 (비교 실험용)
kkma = Kkma()           # 꼬꼬마 분석기
okt = Okt()             # Open Korean Text 분석기  
komoran = Komoran()     # 코모란 분석기
hannanum = Hannanum()   # 한나눔 분석기
kiwi = Kiwi()           # Kiwi 분석기

In [ ]:
# 형태소 분석 비교를 위한 테스트 문장
text = "안녕하세요? 형태소 분석기 테스트베드입니다."

In [ ]:
# === 5가지 형태소 분석기 성능 비교 ===

print("kkma    : \t", end="")
print(" ".join(kkma.morphs(text)))        # 꼬꼬마 결과

print("okt     : \t", end="")  
print(" ".join(okt.morphs(text)))         # Okt 결과

print("komoran : \t", end="")
print(" ".join(komoran.morphs(text)))     # 코모란 결과

print("hannanum: \t", end="")
print(" ".join(hannanum.morphs(text)))    # 한나눔 결과  

print("kiwi    : \t", end="")
# Kiwi는 토큰 객체를 반환하므로 form 속성만 추출
print(" ".join([tok.form for tok in kiwi.tokenize(text)]))

In [ ]:
# Kiwi 사용자 단어 추가 기능 테스트
kiwi.add_user_word("안녕하세요 반가워요", "NNP", 0)  # 고유명사로 사용자 정의 단어 추가

In [ ]:
# 사용자 정의 단어 추가 후 토큰화 결과 확인
print("kiwi    : \t", end="")
print(" ".join([tok.form for tok in kiwi.tokenize(text)]))

In [ ]:
# 꼬꼬마 분석기용 토큰화 함수
def kkma_tokenize(text):
    """꼬꼬마를 사용한 토큰화"""
    return [token for token in kkma.morphs(text)]

In [ ]:
# Okt 분석기용 토큰화 함수
def okt_tokenize(text):
    """Okt를 사용한 토큰화"""
    return [token for token in okt.morphs(text)]

In [ ]:
# === 꼬꼬마 기반 검색기 추가 구성 ===

# 꼬꼬마 BM25 단독 검색기
kkma_bm25 = BM25Retriever.from_documents(docs, preprocess_func=kkma_tokenize)

# 꼬꼬마 + FAISS 앙상블 (키워드 우선: 7:3)
kkma_bm25_faiss_73 = EnsembleRetriever(
    retrievers=[kkma_bm25, faiss],  # 꼬꼬마 BM25 + FAISS
    weights=[0.7, 0.3],  # 꼬꼬마 키워드 검색에 높은 가중치
    search_type="mmr",
)

# 꼬꼬마 + FAISS 앙상블 (의미 우선: 3:7)  
kkma_bm25_faiss_37 = EnsembleRetriever(
    retrievers=[kkma_bm25, faiss],
    weights=[0.3, 0.7],  # FAISS 의미 검색에 높은 가중치
    search_type="mmr",
)

# === Okt 기반 검색기 추가 구성 ===

# Okt BM25 단독 검색기
okt_bm25 = BM25Retriever.from_documents(docs, preprocess_func=okt_tokenize)

# Okt + FAISS 앙상블 (키워드 우선: 7:3)
okt_bm25_faiss_73 = EnsembleRetriever(
    retrievers=[okt_bm25, faiss],  # Okt BM25 + FAISS
    weights=[0.7, 0.3],  # Okt 키워드 검색에 높은 가중치
    search_type="mmr",
)

# Okt + FAISS 앙상블 (의미 우선: 3:7)
okt_bm25_faiss_37 = EnsembleRetriever(
    retrievers=[okt_bm25, faiss],
    weights=[0.3, 0.7],  # FAISS 의미 검색에 높은 가중치
    search_type="mmr",
)

# === 전체 검색기 모음 (총 13개 검색기) ===
retrievers = {
    # 기존 7개 검색기
    "bm25": bm25,
    "kiwi_bm25": kiwi_bm25,
    "faiss": faiss,
    "bm25_faiss_73": bm25_faiss_73,
    "bm25_faiss_37": bm25_faiss_37,
    "kiwi_bm25_faiss_73": kiwibm25_faiss_73,
    "kiwi_bm25_faiss_37": kiwibm25_faiss_37,
    
    # 추가된 6개 검색기 (꼬꼬마, Okt 기반)
    "kkma_bm25": kkma_bm25,
    "kkma_bm25_faiss_73": kkma_bm25_faiss_73,
    "kkma_bm25_faiss_37": kkma_bm25_faiss_37,
    "okt_bm25": okt_bm25,
    "okt_bm25_faiss_73": okt_bm25_faiss_73,
    "okt_bm25_faiss_37": okt_bm25_faiss_37,
}

In [ ]:
# === 최종 종합 검색 테스트 (총 13가지 검색기 비교) ===

# 동일한 6가지 쿼리로 모든 검색기 성능 비교
print_search_results(retrievers, "금융보험")        # 기본 복합어 검색
print_search_results(retrievers, "금융 보험")       # 공백 분리 검색  
print_search_results(retrievers, "금융저축보험")    # 긴 복합어 검색
print_search_results(retrievers, "축산물 보험")     # 특수 상품명 검색
print_search_results(retrievers, "저축금융보험")    # 단어 순서 변경 검색
print_search_results(retrievers, "금융보씨 개인정보 조회")  # 노이즈 포함 검색

---

# 종합 결론 및 실무 적용 가이드

## 성능 분석 결과

13가지 검색기 비교 실험을 통해 도출된 주요 인사이트를 정리합니다.

### 형태소 분석기별 종합 평가

| 순위 | 분석기 | 종합 점수 | 주요 강점 | 권장 사용처 |
|------|--------|-----------|-----------|------------|
| **1위** | **Kiwi** | ⭐⭐⭐⭐⭐ | 빠른 속도, 높은 정확도, 한글 최적화 | 실무 프로젝트 최우선 권장 |
| **2위** | **Okt** | ⭐⭐⭐⭐ | 균형잡힌 성능, 높은 안정성 | 안정성이 중요한 시스템 |
| **3위** | **Kkma** | ⭐⭐⭐ | 가장 정밀한 분석 | 정확성이 속도보다 중요한 연구 |

### 검색 전략별 권장 사항

#### 서비스 유형별 최적 조합

| 서비스 유형 | 권장 검색기 | 가중치 | 선택 이유 |
|-------------|-------------|--------|----------|
| **전자상거래 상품 검색** | Kiwi-BM25 + FAISS | 7:3 | 정확한 상품명 매칭 + 빠른 속도 |
| **문서 검색 시스템** | Kiwi-BM25 + FAISS | 3:7 | 문맥적 이해 + 한글 최적화 |
| **학술 논문 검색** | Kkma-BM25 + FAISS | 5:5 | 정밀한 용어 분석 + 의미 파악 |
| **실시간 챗봇 검색** | Kiwi-BM25 | 단독 | 최대 속도 우선 |
| **법률/의료 전문 검색** | Kkma-BM25 + FAISS | 7:3 | 전문 용어 정확성 우선 |

### 성능 최적화 고려사항

#### 시스템 자원 요구사항

| 검색기 구성 | CPU 사용량 | 메모리 사용량 | 초기화 시간 | 처리 속도 |
|-------------|------------|---------------|-------------|-----------|
| **BM25 단독** | 낮음 | 낮음 | 빠름 | 빠름 |
| **Kiwi-BM25 단독** | 보통 | 보통 | 보통 | 빠름 |
| **FAISS 단독** | 높음 | 높음 | 느림 | 보통 |
| **Ensemble 조합** | 높음 | 높음 | 느림 | 보통 |

## 실무 적용 가이드

### 구현 단계별 체크리스트

#### Phase 1: 요구사항 분석
- [ ] 검색 대상 문서의 특성 분석 (도메인, 길이, 형식)
- [ ] 예상 검색 쿼리 유형 분석 (키워드, 문장, 복합어)
- [ ] 성능 요구사항 정의 (응답시간, 정확도, 처리량)
- [ ] 시스템 자원 제약사항 파악 (CPU, 메모리, 저장공간)

#### Phase 2: 검색기 선택 및 구성
- [ ] 후보 검색기 조합 선정 (2-3개)
- [ ] 샘플 데이터로 성능 테스트 수행
- [ ] 가중치 및 매개변수 최적화
- [ ] 프로덕션 환경 적합성 검증

#### Phase 3: 배포 및 모니터링
- [ ] 성능 모니터링 시스템 구축
- [ ] A/B 테스트를 통한 사용자 만족도 측정
- [ ] 검색 품질 지표 정의 및 추적
- [ ] 정기적인 모델 업데이트 계획 수립

### 문제 해결 가이드

#### 일반적인 문제와 해결책

| 문제 상황 | 가능한 원인 | 권장 해결책 |
|-----------|-------------|-------------|
| **검색 정확도 낮음** | 형태소 분석기 부적합 | Kiwi로 변경, 사용자 사전 추가 |
| **응답 속도 느림** | Ensemble 복잡도 과다 | 단일 검색기 사용, 인덱싱 최적화 |
| **메모리 사용량 과다** | 벡터 저장소 크기 | 차원 축소, 배치 처리 적용 |
| **신조어 검색 실패** | 사전 업데이트 부족 | 정기적 사용자 사전 업데이트 |

## 향후 학습 방향

### 심화 학습 주제

1. **커스텀 토큰화**: 도메인 특화 전처리 기법
2. **하이브리드 검색**: 더 정교한 앙상블 전략
3. **성능 튜닝**: 대규모 데이터 처리 최적화
4. **품질 평가**: 정량적 검색 품질 측정 지표

### 실습 과제

1. **개인 프로젝트**: 자신만의 데이터셋으로 검색 시스템 구축
2. **파라미터 튜닝**: 다양한 가중치 비율(6:4, 8:2, 9:1) 실험
3. **도메인 적응**: 특정 분야(의료, 법률, 기술) 전문 검색 시스템 개발
4. **사용자 피드백**: 실제 사용자 평가를 통한 시스템 개선

---

## 학습 완료

한글 검색 시스템 개발에 필요한 핵심 기술들을 성공적으로 학습하였습니다.

이제 **형태소 분석부터 하이브리드 검색까지**의 전체 파이프라인을 이해하고, 실무 프로젝트에 즉시 적용할 수 있는 수준에 도달했습니다.

### 주요 성취

- 5가지 형태소 분석기의 특성과 적용 분야 이해
- 13가지 검색 전략의 성능 특성 분석
- 실무 환경별 최적 검색기 조합 선택 능력 획득
- 프로덕션 시스템 구축을 위한 실무 가이드라인 습득

지속적인 실험과 개선을 통해 더욱 정교한 한글 검색 시스템을 구축해 나가시기 바랍니다.